In [1]:
# Speech emotion recognition on MSP-IMPROV dataset
# feature: egemaps
# Algorithm: as described in Bagus TA et al., "RNN-based dimenisional SER",
#                            ASJ Autumn 2019.
# 2019-12-24: Modified from ser_improv_egamaps, change input to HFS
#             Filter only natural interaction and improvisation data
# 2019-12-25: Updaate to adapt train/test data-label, change it accordingly
#             to be run in csl directory (although data in improv_loso/)
# 2020-05-26: update for SC paper

import numpy as np
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Flatten, \
                         Dropout, Flatten, LSTM, BatchNormalization, \
                         RNN, SimpleRNN, Activation
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import mean_squared_error
import random as rn
import tensorflow as tf
# from attention_helper import AttentionDecoder
#from read_csv import load_features
# CSL Paper: Dimensional speech emotion recognition from acoustic and text
# Changelog:
# 2019-09-01: initial version
# 2019-10-06: optimizer MTL parameters with linear search (in progress)
# 2019-12-21: update file for CSL journal
#             this python use following feature: mean, std, silence
import codecs
import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.manifold import TSNE

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

#from keras_self_attention import SeqSelfAttention
from keras.callbacks import EarlyStopping, ModelCheckpoint
# from attention_helper import AttentionDecoder
#from read_csv import load_features

import random as rn
import tensorflow as tf

import gensim
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors

Using TensorFlow backend.
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:
# loading file and label, choose the feature type first!
feat = np.load('../improv_loso/data/feat_hfs_mspin_train.npy')
feat_train = feat.reshape(feat.shape[0], 1, feat.shape[1])

feat_test = np.load('../improv_loso/data/feat_hfs_mspin_test.npy')
feat_test = feat_test.reshape(feat_test.shape[0], 1, feat_test.shape[1])

# load train and test ata
list_path_train = '../improv_loso/data/data_mspin_train.csv'
list_file = pd.read_csv(list_path_train, index_col=None)

vad_list = [list_file['v'], list_file['a'], list_file['d']]
vad_train = np.array(vad_list).T

list_path_test = '../improv_loso/data/data_mspin_test.csv'
list_test = pd.read_csv(list_path_test, index_col=None)

vad_test = [list_test ['v'], list_test['a'], list_test['d']]
vad_test = np.array(vad_test).T

feat = np.concatenate([feat_train, feat_test], axis=0)
vad = np.concatenate([vad_train, vad_test])

# uncomment the following to remove silence feature (HFS1)
feat = feat[:,:, :-1]

# load test data
print('Feature shape: ', feat.shape)
print('Label shape: ', vad.shape)


Feature shape:  (7166, 1, 46)
Label shape:  (7166, 3)


In [4]:
# load data - label
list_path_train = '../improv_loso/data/data_mspin_train.csv'
list_train = pd.read_csv(list_path_train, index_col=None)

# vad_train = [list_train['v'], list_train['a'], list_train['d']]
# vad_train = np.array(vad_train).T

list_path_test = '../improv_loso/data/data_mspin_test.csv'
list_test = pd.read_csv(list_path_test, index_col=None)

## Text processing train
list_file = pd.concat([list_train, list_test])
text = list_file['transcript']
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
word_index = tokenizer.word_index
nb_words = len(word_index) + 1
print('Found %s unique tokens' % len(word_index))

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_text = []
x_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

# x_text = np.load('improv_loso/data/x_text.npy')
# x_train_text = x_text[:5744] 
# x_test_text = x_text[5744:]

# vad = np.load('improv_loso/data/vad.npy')
# vad_train = vad[:5744]
# vad_test = vad[5744:]

EMBEDDING_DIM = 300
# nb_words = 3438
# MAX_SEQUENCE_LENGTH = 300

#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
file_loc =  '../../../dimensionalSM/data/GoogleNews-vectors-negative300.bin'
print (file_loc)

word_vectors = KeyedVectors.load_word2vec_format(file_loc, binary=True)

g_word_embedding_matrix = {}
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=nb_words:
        continue
    try:
        gembedding_vector = word_vectors[word]
        g_word_embedding_matrix[i] = gembedding_vector
    except KeyError:
        g_word_embedding_matrix[i] = np.random.normal(0, np.sqrt(0.25), EMBEDDING_DIM)

del(word_vectors)

# print input size
print('Feature shape: ', x_text.shape)
# print('Label shape: ', vad_train.shape)


Found 3427 unique tokens
../../../dimensionalSM/data/GoogleNews-vectors-negative300.bin
Feature shape:  (7166, 300)


In [5]:
## standardization
scaled_vad = True

if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1,  keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss

In [6]:
# split train/test
split = 5733

In [7]:
def model():
    # speech network
    input_speech = Input(shape=(feat.shape[1], feat.shape[2]), name='speech_input')
    net_speech = BatchNormalization()(input_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(128, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    model_speech = Flatten()(net_speech)
    #model_speech = Dropout(0.1)(net_speech)
    
    #text network
    input_text = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net_text = Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(input_text)
    net = Bidirectional(LSTM(128, return_sequences=True))(net_text)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    model_text = Dense(64)(net)

    # combined model
    model_combined = concatenate([model_speech, model_text])
    model_combined = Dense(64, activation='relu')(model_combined)
    model_combined = Dense(32, activation='relu')(model_combined)
    #model_combined = Dropout(0.4)(model_combined)
    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_combined) for name in target_names]

    model = Model([input_speech, input_text], model_combined) 
    model.compile(loss=ccc_loss,
                  loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model
# model2 = model()

In [8]:
# main function for for LOSO
def main(time):
    model_1 = model()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [feat[:split],x_text[:split]],
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [feat[split:],x_text[split:]], vad[split:].T.tolist())
    print(metrik[-3:])
    # make prediction
    predict = model_1.predict([feat[4586:],x_text[4586:]], batch_size=32)
    np.save('5733npy-W2V/st_5733W2V-npy'+str(time)+'.npy',  
             np.array(predict).reshape(3, 2580).T)
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('5733npy-W2V/st_5733-W2V.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 4586 samples, validate on 1147 samples
Epoch 1/50
4586/4586 [==============================] - 93s 20ms/step - loss: 1.7733 - v_loss: 0.6810 - a_loss: 0.5145 - d_loss: 0.5784 - v_ccc: 0.3187 - a_ccc: 0.4852 - d_ccc: 0.4228 - val_loss: 1.8258 - val_v_loss: 0.7324 - val_a_loss: 0.5330 - val_d_loss: 0.5599 - val_v_ccc: 0.2666 - val_a_ccc: 0.4675 - val_d_ccc: 0.4401
Epoch 2/50
4586/4586 [==============================] - 87s 19ms/step - loss: 1.3132 - v_loss: 0.4816 - a_loss: 0.3705 - d_loss: 0.4633 - v_ccc: 0.5202 - a_ccc: 0.6303 - d_ccc: 0.5363 - val_loss: 1.5393 - val_v_loss: 0.6028 - val_a_loss: 0.4183 - val_d_loss: 0.5175 - val_v_ccc: 0.3962 - val_a_ccc: 0.5823 - val_d_ccc: 0.4822
Epoch 3/50
4586/4586 [==============================] - 88s 19ms/step - loss: 1.1741 - v_loss: 0.4015 - a_loss: 0.3442 - d_loss: 0.4256 - v_ccc: 0.5976 - a_ccc: 0.6554 - d_ccc: 0.5730 - val_loss: 1.5559 - val_v_loss: 0

Epoch 12/50
4586/4586 [==============================] - 88s 19ms/step - loss: 0.5547 - v_loss: 0.1710 - a_loss: 0.1841 - d_loss: 0.2050 - v_ccc: 0.8298 - a_ccc: 0.8187 - d_ccc: 0.7967 - val_loss: 1.8341 - val_v_loss: 0.6926 - val_a_loss: 0.4943 - val_d_loss: 0.6466 - val_v_ccc: 0.3061 - val_a_ccc: 0.5060 - val_d_ccc: 0.3537
Epoch 13/50
1433/1433 [==============================] - 8s 5ms/step
[0.31740638613700867, 0.5263243317604065, 0.36418208479881287]
1
Train on 4586 samples, validate on 1147 samples
Epoch 1/50
4586/4586 [==============================] - 91s 20ms/step - loss: 1.7911 - v_loss: 0.6642 - a_loss: 0.5338 - d_loss: 0.5956 - v_ccc: 0.3362 - a_ccc: 0.4672 - d_ccc: 0.4054 - val_loss: 1.6204 - val_v_loss: 0.6129 - val_a_loss: 0.4694 - val_d_loss: 0.5376 - val_v_ccc: 0.3861 - val_a_ccc: 0.5312 - val_d_ccc: 0.4623
Epoch 2/50
4586/4586 [==============================] - 86s 19ms/step - loss: 1.3232 - v_loss: 0.4915 - a_loss: 0.3766 - d_loss: 0.4532 - v_ccc: 0.5071 - a_ccc: 0.62

4586/4586 [==============================] - 86s 19ms/step - loss: 0.6321 - v_loss: 0.1964 - a_loss: 0.2016 - d_loss: 0.2336 - v_ccc: 0.8033 - a_ccc: 0.7986 - d_ccc: 0.7661 - val_loss: 1.7193 - val_v_loss: 0.6820 - val_a_loss: 0.4596 - val_d_loss: 0.5771 - val_v_ccc: 0.3172 - val_a_ccc: 0.5408 - val_d_ccc: 0.4228
Epoch 11/50
4586/4586 [==============================] - 87s 19ms/step - loss: 0.5831 - v_loss: 0.1820 - a_loss: 0.1878 - d_loss: 0.2131 - v_ccc: 0.8180 - a_ccc: 0.8122 - d_ccc: 0.7868 - val_loss: 1.8546 - val_v_loss: 0.6783 - val_a_loss: 0.4941 - val_d_loss: 0.6808 - val_v_ccc: 0.3202 - val_a_ccc: 0.5059 - val_d_ccc: 0.3193
Epoch 12/50
1433/1433 [==============================] - 8s 5ms/step
[0.2708744406700134, 0.4910198748111725, 0.3770866096019745]
3
Train on 4586 samples, validate on 1147 samples
Epoch 1/50
4586/4586 [==============================] - 90s 20ms/step - loss: 1.7898 - v_loss: 0.6773 - a_loss: 0.5211 - d_loss: 0.5888 - v_ccc: 0.3225 - a_ccc: 0.4781 - d_ccc: 0

4586/4586 [==============================] - 86s 19ms/step - loss: 0.6522 - v_loss: 0.1973 - a_loss: 0.2117 - d_loss: 0.2456 - v_ccc: 0.8028 - a_ccc: 0.7900 - d_ccc: 0.7551 - val_loss: 1.6503 - val_v_loss: 0.6287 - val_a_loss: 0.4500 - val_d_loss: 0.5704 - val_v_ccc: 0.3699 - val_a_ccc: 0.5501 - val_d_ccc: 0.4298
Epoch 11/50
4586/4586 [==============================] - 87s 19ms/step - loss: 0.5956 - v_loss: 0.1801 - a_loss: 0.1936 - d_loss: 0.2236 - v_ccc: 0.8205 - a_ccc: 0.8069 - d_ccc: 0.7771 - val_loss: 1.6385 - val_v_loss: 0.6476 - val_a_loss: 0.4469 - val_d_loss: 0.5428 - val_v_ccc: 0.3509 - val_a_ccc: 0.5532 - val_d_ccc: 0.4575
Epoch 12/50
1433/1433 [==============================] - 8s 6ms/step
[0.30202046036720276, 0.5392580628395081, 0.42563721537590027]
5
Train on 4586 samples, validate on 1147 samples
Epoch 1/50
4586/4586 [==============================] - 92s 20ms/step - loss: 1.7672 - v_loss: 0.6779 - a_loss: 0.5122 - d_loss: 0.5801 - v_ccc: 0.3240 - a_ccc: 0.4886 - d_ccc:

4586/4586 [==============================] - 88s 19ms/step - loss: 0.7003 - v_loss: 0.2155 - a_loss: 0.2224 - d_loss: 0.2620 - v_ccc: 0.7847 - a_ccc: 0.7771 - d_ccc: 0.7379 - val_loss: 1.6664 - val_v_loss: 0.7016 - val_a_loss: 0.4395 - val_d_loss: 0.5248 - val_v_ccc: 0.2976 - val_a_ccc: 0.5608 - val_d_ccc: 0.4752
Epoch 10/50
4586/4586 [==============================] - 87s 19ms/step - loss: 0.6526 - v_loss: 0.1998 - a_loss: 0.2090 - d_loss: 0.2450 - v_ccc: 0.8000 - a_ccc: 0.7915 - d_ccc: 0.7560 - val_loss: 1.6364 - val_v_loss: 0.6628 - val_a_loss: 0.4400 - val_d_loss: 0.5330 - val_v_ccc: 0.3362 - val_a_ccc: 0.5602 - val_d_ccc: 0.4673
Epoch 11/50
4586/4586 [==============================] - 88s 19ms/step - loss: 0.5968 - v_loss: 0.1860 - a_loss: 0.1888 - d_loss: 0.2236 - v_ccc: 0.8144 - a_ccc: 0.8119 - d_ccc: 0.7768 - val_loss: 1.8392 - val_v_loss: 0.7065 - val_a_loss: 0.4975 - val_d_loss: 0.6343 - val_v_ccc: 0.2927 - val_a_ccc: 0.5024 - val_d_ccc: 0.3657
Epoch 12/50
1433/1433 [========

...

4586/4586 [==============================] - 86s 19ms/step - loss: 0.9115 - v_loss: 0.2838 - a_loss: 0.2795 - d_loss: 0.3502 - v_ccc: 0.7171 - a_ccc: 0.7205 - d_ccc: 0.6509 - val_loss: 1.6186 - val_v_loss: 0.6333 - val_a_loss: 0.4717 - val_d_loss: 0.5126 - val_v_ccc: 0.3652 - val_a_ccc: 0.5287 - val_d_ccc: 0.4874
Epoch 7/50
4586/4586 [==============================] - 85s 19ms/step - loss: 0.8354 - v_loss: 0.2507 - a_loss: 0.2650 - d_loss: 0.3262 - v_ccc: 0.7489 - a_ccc: 0.7398 - d_ccc: 0.6759 - val_loss: 1.6993 - val_v_loss: 0.6818 - val_a_loss: 0.5070 - val_d_loss: 0.5090 - val_v_ccc: 0.3166 - val_a_ccc: 0.4930 - val_d_ccc: 0.4911
Epoch 8/50
4586/4586 [==============================] - 86s 19ms/step - loss: 0.7612 - v_loss: 0.2334 - a_loss: 0.2339 - d_loss: 0.2962 - v_ccc: 0.7674 - a_ccc: 0.7663 - d_ccc: 0.7051 - val_loss: 1.7406 - val_v_loss: 0.6686 - val_a_loss: 0.4572 - val_d_loss: 0.6142 - val_v_ccc: 0.3300 - val_a_ccc: 0.5429 - val_d_ccc: 0.3865
Epoch 9/50
4586/4586 [===========

4586/4586 [==============================] - 87s 19ms/step - loss: 1.1845 - v_loss: 0.3998 - a_loss: 0.3492 - d_loss: 0.4377 - v_ccc: 0.5998 - a_ccc: 0.6537 - d_ccc: 0.5620 - val_loss: 1.6331 - val_v_loss: 0.6804 - val_a_loss: 0.4384 - val_d_loss: 0.5140 - val_v_ccc: 0.3189 - val_a_ccc: 0.5624 - val_d_ccc: 0.4857
Epoch 4/50
4586/4586 [==============================] - 87s 19ms/step - loss: 1.0724 - v_loss: 0.3439 - a_loss: 0.3180 - d_loss: 0.4110 - v_ccc: 0.6555 - a_ccc: 0.6828 - d_ccc: 0.5894 - val_loss: 1.5514 - val_v_loss: 0.6331 - val_a_loss: 0.4226 - val_d_loss: 0.4945 - val_v_ccc: 0.3657 - val_a_ccc: 0.5777 - val_d_ccc: 0.5053
Epoch 5/50
4586/4586 [==============================] - 87s 19ms/step - loss: 0.9751 - v_loss: 0.3007 - a_loss: 0.2966 - d_loss: 0.3774 - v_ccc: 0.6986 - a_ccc: 0.7030 - d_ccc: 0.6234 - val_loss: 1.7782 - val_v_loss: 0.6821 - val_a_loss: 0.5115 - val_d_loss: 0.5825 - val_v_ccc: 0.3162 - val_a_ccc: 0.4885 - val_d_ccc: 0.4172
Epoch 6/50
4586/4586 [===========

1433/1433 [==============================] - 8s 6ms/step
[0.23654095828533173, 0.5143393874168396, 0.3709864616394043]
14
Train on 4586 samples, validate on 1147 samples
Epoch 1/50
4586/4586 [==============================] - 96s 21ms/step - loss: 1.8047 - v_loss: 0.6736 - a_loss: 0.5309 - d_loss: 0.5948 - v_ccc: 0.3239 - a_ccc: 0.4679 - d_ccc: 0.4035 - val_loss: 1.9854 - val_v_loss: 0.6665 - val_a_loss: 0.6282 - val_d_loss: 0.6907 - val_v_ccc: 0.3328 - val_a_ccc: 0.3723 - val_d_ccc: 0.3096
Epoch 2/50
4586/4586 [==============================] - 86s 19ms/step - loss: 1.3263 - v_loss: 0.4882 - a_loss: 0.3790 - d_loss: 0.4631 - v_ccc: 0.5137 - a_ccc: 0.6221 - d_ccc: 0.5380 - val_loss: 1.6260 - val_v_loss: 0.7138 - val_a_loss: 0.4241 - val_d_loss: 0.4866 - val_v_ccc: 0.2846 - val_a_ccc: 0.5763 - val_d_ccc: 0.5131
Epoch 3/50
4586/4586 [==============================] - 85s 19ms/step - loss: 1.1814 - v_loss: 0.4092 - a_loss: 0.3444 - d_loss: 0.4256 - v_ccc: 0.5899 - a_ccc: 0.6553 - d_ccc: 0

4586/4586 [==============================] - 88s 19ms/step - loss: 0.5583 - v_loss: 0.1701 - a_loss: 0.1767 - d_loss: 0.2147 - v_ccc: 0.8308 - a_ccc: 0.8235 - d_ccc: 0.7874 - val_loss: 1.8069 - val_v_loss: 0.7119 - val_a_loss: 0.4932 - val_d_loss: 0.6008 - val_v_ccc: 0.2871 - val_a_ccc: 0.5069 - val_d_ccc: 0.3991
Epoch 13/50
1433/1433 [==============================] - 8s 6ms/step
[0.27877041697502136, 0.523040235042572, 0.3919701874256134]
16
Train on 4586 samples, validate on 1147 samples
Epoch 1/50
4586/4586 [==============================] - 95s 21ms/step - loss: 1.7856 - v_loss: 0.6770 - a_loss: 0.5234 - d_loss: 0.5831 - v_ccc: 0.3232 - a_ccc: 0.4757 - d_ccc: 0.4156 - val_loss: 1.5799 - val_v_loss: 0.6216 - val_a_loss: 0.4382 - val_d_loss: 0.5197 - val_v_ccc: 0.3778 - val_a_ccc: 0.5625 - val_d_ccc: 0.4799
Epoch 2/50
4586/4586 [==============================] - 85s 18ms/step - loss: 1.3410 - v_loss: 0.4905 - a_loss: 0.3824 - d_loss: 0.4688 - v_ccc: 0.5098 - a_ccc: 0.6182 - d_ccc: 0

4586/4586 [==============================] - 90s 20ms/step - loss: 0.6448 - v_loss: 0.1963 - a_loss: 0.2073 - d_loss: 0.2436 - v_ccc: 0.8048 - a_ccc: 0.7923 - d_ccc: 0.7582 - val_loss: 1.8053 - val_v_loss: 0.7036 - val_a_loss: 0.4917 - val_d_loss: 0.6089 - val_v_ccc: 0.2948 - val_a_ccc: 0.5085 - val_d_ccc: 0.3914
Epoch 11/50
4586/4586 [==============================] - 89s 19ms/step - loss: 0.5876 - v_loss: 0.1787 - a_loss: 0.1935 - d_loss: 0.2151 - v_ccc: 0.8210 - a_ccc: 0.8069 - d_ccc: 0.7845 - val_loss: 1.8279 - val_v_loss: 0.6797 - val_a_loss: 0.5294 - val_d_loss: 0.6174 - val_v_ccc: 0.3190 - val_a_ccc: 0.4703 - val_d_ccc: 0.3827
Epoch 12/50
1433/1433 [==============================] - 8s 6ms/step
[0.2712343633174896, 0.48549118638038635, 0.3717597723007202]
18
Train on 4586 samples, validate on 1147 samples
Epoch 1/50
4586/4586 [==============================] - 99s 22ms/step - loss: 1.8059 - v_loss: 0.6846 - a_loss: 0.5362 - d_loss: 0.5830 - v_ccc: 0.3158 - a_ccc: 0.4624 - d_ccc:

In [9]:
# earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True)
# hist = model2.fit([feat[:split],x_text[:split]], vad[:split].T.tolist(), batch_size=32,
#                   validation_split=0.2, epochs=50, verbose=1, shuffle=True,
#                   callbacks=[earlystop])

# # outs =loss, mse_v, mse_a, mse_d, ccc_v, ccc_a, ccc_d
# # test data start from 5744/split
# metrik2 = model2.evaluate([feat[split:],x_text[split:]], vad[split:].T.tolist())#, a[8000:], d[8000:]])
# print(metrik2[-3:])

# # # make prediction
# # feat_predict = feat[4596:]
# # vad_predict = vad[4596:]

# predict2 = model2.predict([feat[4596:],x_text[4596:]], batch_size=32)
# np.save('st_glove_npy_loso',  np.array(predict2).reshape(3, 2570).T)